In [ ]:
# N-BEAT WITH 18 MONTHS BTC DATA WITH SENTIMENT
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda, subtract, add
from tensorflow.keras.models import Model

# ----------------------------
# 1. Data Preparation (Multivariate: 'close' and 'avg_sentiment')
# ----------------------------

# Load BTC data and sentiment data
 # Adjust the path as needed

# Sort and filter BTC data
df_btc.sort_values('datetime', inplace=True)
df_btc = df_btc[df_btc['datetime'] <= '2019-05-01'].copy()
df_btc = df_btc[['datetime', 'close']]  # Remove volume

# Merge BTC data with sentiment data on datetime
merged_df = pd.merge(df_btc, df_sentiment[['datetime', 'avg_sentiment']], on='datetime', how='inner')
print(merged_df.head())

# Split the merged data into training and testing sets
train_df = merged_df[(merged_df['datetime'] >= '2018-01-01') & (merged_df['datetime'] < '2019-02-01')].copy()
test_df = merged_df[merged_df['datetime'] >= '2019-02-01'].copy()

# Create separate scalers for 'close' and 'avg_sentiment'
close_scaler = MinMaxScaler()
sentiment_scaler = MinMaxScaler()

# Fit scalers on training data and transform
train_df['close_scaled'] = close_scaler.fit_transform(train_df[['close']])
train_df['sentiment_scaled'] = sentiment_scaler.fit_transform(train_df[['avg_sentiment']])
test_df['close_scaled'] = close_scaler.transform(test_df[['close']])
test_df['sentiment_scaled'] = sentiment_scaler.transform(test_df[['avg_sentiment']])

# ----------------------------
# 2. Creating Sequences for N-BEATS (Multivariate with Sentiment)
# ----------------------------
lookback = 30  # number of days to look back

def create_sequences_multivariate(data, lookback):
    """
    data: numpy array with shape (n_samples, n_features)
    Returns sequences with shape (n_sequences, lookback, n_features) and targets.
    The target is the close price (first column) at the current time step.
    """
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i - lookback:i])
        y.append(data[i, 0])  # first column: close_scaled
    return np.array(X), np.array(y)

# Combine the two features in the order: [close_scaled, sentiment_scaled]
train_data = train_df[['close_scaled', 'sentiment_scaled']].values
test_data = test_df[['close_scaled', 'sentiment_scaled']].values

X_train, y_train = create_sequences_multivariate(train_data, lookback)
X_test, y_test = create_sequences_multivariate(test_data, lookback)

# For N-BEATS, we flatten the sequences into vectors.
n_features = X_train.shape[2]  # should be 2 now
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Define backcast length as flattened input dimension and forecast length as 1 (for close price)
backcast_length = lookback * n_features
forecast_length = 1

# ----------------------------
# 3. Building a Simplified N-BEATS Model in Keras (with Sentiment)
# ----------------------------
def nbeats_block(backcast_length, forecast_length, n_neurons=128, n_layers=3):
    """
    Creates a single N-BEATS block.
    """
    input_block = Input(shape=(backcast_length,))
    x = input_block
    for _ in range(n_layers):
        x = Dense(n_neurons, activation='relu')(x)
    theta = Dense(backcast_length + forecast_length)(x)
    # Split theta into backcast and forecast parts
    backcast = Lambda(lambda x: x[:, :backcast_length])(theta)
    forecast = Lambda(lambda x: x[:, backcast_length:])(theta)
    return Model(inputs=input_block, outputs=[backcast, forecast])

# Stack several N-BEATS blocks
n_blocks = 3

input_layer = Input(shape=(backcast_length,))
residual = input_layer  # initialize the residual as the input
forecast_outputs = []

for i in range(n_blocks):
    block = nbeats_block(backcast_length, forecast_length, n_neurons=128, n_layers=3)
    backcast, forecast = block(residual)
    residual = subtract([residual, backcast])
    forecast_outputs.append(forecast)

# Sum forecast outputs from all blocks for the final forecast
final_forecast = add(forecast_outputs)
model_nbeats = Model(inputs=input_layer, outputs=final_forecast)
model_nbeats.compile(optimizer='adam', loss='mse')
model_nbeats.summary()

# ----------------------------
# 4. Training the N-BEATS Model
# ----------------------------
history_nbeats = model_nbeats.fit(
    X_train_flat, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

# ----------------------------
# 5. Prediction, Inverse Transformation, and Evaluation
# ----------------------------
predictions_nbeats_scaled = model_nbeats.predict(X_test_flat)
# Inverse transform predictions to get actual close price values
predictions_nbeats = close_scaler.inverse_transform(predictions_nbeats_scaled)
y_test_actual_nbeats = close_scaler.inverse_transform(y_test.reshape(-1, 1))

# Align test dates (we lose the first 'lookback' days due to sequence creation)
test_dates = test_df['datetime'].values[lookback:]

plt.figure(figsize=(12, 6))
plt.plot(test_dates, y_test_actual_nbeats, label='Actual Close Price', color='blue')
plt.plot(test_dates, predictions_nbeats, label='Predicted Close Price (N-BEATS with Sentiment)', color='red', linestyle='--')
plt.xlabel('Date')
plt.ylabel('BTC Close Price')
plt.title('BTC Close Price Prediction using N-BEATS (with Sentiment): Actual vs Predicted')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Calculate performance metrics
mse_nbeats = mean_squared_error(y_test_actual_nbeats, predictions_nbeats)
mae_nbeats = mean_absolute_error(y_test_actual_nbeats, predictions_nbeats)
rmse_nbeats = np.sqrt(mse_nbeats)
mape_nbeats = mean_absolute_percentage_error(y_test_actual_nbeats, predictions_nbeats) * 100
r2_nbeats = r2_score(y_test_actual_nbeats, predictions_nbeats)

print("Performance Metrics for the N-BEATS Model (with Sentiment):")
print(f"MSE: {mse_nbeats:.2f}")
print(f"MAE: {mae_nbeats:.2f}")
print(f"RMSE: {rmse_nbeats:.2f}")
print(f"MAPE: {mape_nbeats:.2f}%")
print(f"R2 Score: {r2_nbeats:.2f}")
